In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df.head()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,df_target, test_size=0.2, random_state=42)
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=100, random_state=0)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
import math

In [ ]:
class CSI_BPSO:

  def __init__(self,objective_function,n_iteration=50,population_size=50,
               c1=2,c2=2,w=0.9):
    self.population_size = population_size
    self.n_iteration=n_iteration
    self.historical_score_record=[]
    self.objective_function=objective_function
    self.c1=c1
    self.c2=c2
    self.w=w

  def initialize_population(self,X):
        self.individuals =  np.random.randint(0,2,size=(self.population_size,X.shape[1]))


  def evaluate_fitness(self,model,X_train,y_train,X_valid,y_valid):
        scores =  []
        for i,individual in enumerate(self.individuals):
            chosen_features = [index for index in range(X_train.shape[1]) if individual[index]==1]
            X_train_copy = X_train.iloc[:,chosen_features]
            X_valid_copy = X_valid.iloc[:,chosen_features]
            score = self.objective_function(model,X_train_copy,y_train,X_valid_copy,y_valid)
            if score< self.current_best_scores[i]:
              self.current_best_scores[i]=score
              self.current_best_individual_score_dimensions[i]=individual

            scores.append(score)
        self.fitness_scores = scores
 

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))
        
  def fit(self,model,X_train,y_train,X_valid,y_valid):
    list_best_fitness=[]
    list_mean_fitness=[]
    list_min_fitness=[]
    self.initialize_population(X_train)

    self.best_score=np.inf
    self.best_dim=np.ones(X_train.shape[1]) 
    values_list=[]

    self.current_best_individual_score_dimensions=self.individuals
    self.current_best_scores = [np.inf]*self.population_size 
    self.gbest_individual=self.best_dim
    self.v=np.zeros((self.population_size,X_train.shape[1]))

    for i in range(self.n_iteration):

      if (self.individuals.sum(axis=1)==0).sum()>0:
        print((self.individuals.sum(axis=1)==0).sum(),' individuals went zero')
        self.individuals[self.individuals.sum(axis=1)==0]=np.random.randint(0,2,(self.individuals[self.individuals.sum(axis=1)==0].shape[0],\
                                                                                self.individuals[self.individuals.sum(axis=1)==0].shape[1]))
      self.evaluate_fitness(model,X_train,y_train,X_valid,y_valid)

      

      if np.array(self.fitness_scores).min()<self.best_score:
        self.best_dim=self.individuals[np.array(self.fitness_scores).argmin()]
        self.best_score=np.array(self.fitness_scores).min()
      self.gbest_individual=self.best_dim

      
      temp=list(X_train.columns[np.where( self.individuals[np.array(self.fitness_scores).argmin()] )])

       
      r1=np.random.random((self.population_size,X_train.shape[1]))
      r2=np.random.random((self.population_size,X_train.shape[1]))

      self.fitness_scores_numpy=np.array(self.fitness_scores)
      self.qi=np.array(self.fitness_scores_numpy-self.fitness_scores_numpy.max())/(self.fitness_scores_numpy.min()-self.fitness_scores_numpy.max())
      self.Mi=self.qi
      self.Mi=2*self.Mi-1
    

      temp=self.individuals
      temp_2=(( (temp.reshape(temp.shape[0],1,temp.shape[1])-temp.reshape(1,temp.shape[0],temp.shape[1])).reshape(temp.shape[0]**2,temp.shape[1])**2) )
      temp_3=temp_2.reshape(temp.shape[0],temp.shape[0],temp.shape[1]).sum(axis=2)

      temp_4=np.where(temp_3<int(X_train.shape[1]/2) )
      interim=np.array(list(zip(temp_4[0],temp_4[1])))
      interim=interim[interim[:,0]!=interim[:,1]]
      temp_4=(interim[:,0],interim[:,1])

      temp_4_R0=np.where(temp_3<int(X_train.shape[1]/3))
      interim=np.array(list(zip(temp_4_R0[0],temp_4_R0[1])))
      interim=interim[interim[:,0]!=interim[:,1]]
      temp_4_R0=(interim[:,0],interim[:,1])
      
     
      self.acc=[]
      self.acc_R0=[]

      self.acc_individual=[]

      Mi=self.Mi
      for ind_index,individual in enumerate(temp):   
        indexs=np.argsort(self.Mi[temp_4[1][np.where(temp_4[0]==ind_index)]] )[:int(len(temp_4[1][np.where(temp_4[0]==ind_index)])/1)]
        indexs_R0=np.argsort(self.Mi[temp_4_R0[1][np.where(temp_4_R0[0]==ind_index)]] )[:int(len(temp_4_R0[1][np.where(temp_4_R0[0]==ind_index)])/1)] 
        
        temp_5=temp[indexs]*np.repeat(Mi[indexs],temp.shape[1]).reshape(Mi[indexs].shape[0],temp.shape[1]) 
        temp_6=temp_5.sum(axis=0)/(temp[indexs].sum(axis=0))
        temp_6[np.isnan(temp_6)] = 0

        acc1=temp_6

   
        self.acc.append(list(temp_6))

        temp_5=temp[indexs_R0]*np.repeat(Mi[indexs_R0],temp.shape[1]).reshape(Mi[indexs_R0].shape[0],temp.shape[1]) 
        temp_6=temp_5.sum(axis=0)/(temp[indexs_R0].sum(axis=0))
        temp_6[np.isnan(temp_6)] = 0
        self.acc_R0.append(list(temp_6))

        temp_6=(0.8*acc1+0.2*temp_6)

        self.acc_individual.append( list(self.individuals[np.sqrt(((self.individuals-temp_6)**2).sum(axis=1)).argmin()]))

      self.acc=np.array(self.acc)
      self.acc_R0=np.array(self.acc_R0)

      values_list.append(self.best_score )
      
      r3=np.random.random((self.population_size,X_train.shape[1]))
      r4=np.random.random((self.population_size,X_train.shape[1]))

      self.v=self.w*(self.v ) +self.c1*r1*(  (self.gbest_individual ) -self.individuals)+\
       self.c2*r2*( (self.current_best_individual_score_dimensions  ) -self.individuals)+2*r3*( np.array(self.acc_individual)-self.individuals )#+2*r4*( np.array(self.acc_individual2)-self.individuals )

      self.v=np.where(self.v>6,6,self.v)
      self.v=np.where(self.v<-6,-6,self.v)

      list_best_fitness.append(self.best_score)
      list_mean_fitness.append(np.array(self.fitness_scores).mean())
      list_min_fitness.append(np.array(self.fitness_scores).min())
      
      self.s_v=self.sigmoid(self.v)

      rand_nums=np.random.random(size=(self.population_size,X_train.shape[1]))
      self.individuals=np.where(rand_nums<self.s_v,1,0)

    return self.best_dim,   list_best_fitness,\
                list_mean_fitness,list_min_fitness